In [ ]:
import pandas as pd                                                                                       # data frame
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV                                             # validacao cruzada e repitida
from imblearn.pipeline import Pipeline                                                                    # aplicar o smote apenas nos subjconjuntos de treinamento
from sklearn.naive_bayes import GaussianNB                                                                # NB
from sklearn.ensemble import RandomForestClassifier                                                       # random forest
from sklearn.tree import DecisionTreeClassifier                                                           # tree
from sklearn.neural_network import MLPClassifier                                                          # MPL
from sklearn.svm import SVC                                                                               # svm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score                     # importa as métricas
from imblearn.combine import SMOTETomek                                                            # TomekLinks

In [ ]:
# lê os dados do arquivo irish.csv
df = pd.read_csv('/content/irish.csv')

# divide x e y
X = df.drop('class', axis=1)
y = df['class']

In [ ]:
pipeline = Pipeline([
    ('sampling', SMOTETomek(sampling_strategy='all', random_state=15)),
    ('gnb', GaussianNB())
])

model = 'Naive Bayes'

In [ ]:
pipeline = Pipeline([
    ('sampling', SMOTETomek(sampling_strategy='all', random_state=15)),
    ('gnb', RandomForestClassifier())
])

model = 'Random Forest'

In [ ]:
pipeline = Pipeline([
    ('sampling', SMOTETomek(sampling_strategy='all', random_state=15)),
    ('gnb', DecisionTreeClassifier())
])

model = 'Decision Tree'

In [ ]:
pipeline = Pipeline([
    ('sampling', SMOTETomek(sampling_strategy='all', random_state=15)),
    ('gnb', MLPClassifier())
])

model = 'Multilayer Perceptron'

In [ ]:
pipeline = Pipeline([
    ('sampling', SMOTETomek(sampling_strategy='all', random_state=15)),
    ('gnb', SVC())
])

model = 'SVM'

In [ ]:
# validadação cruzada
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=15)

In [ ]:
# printa as classes
classes = sorted(y.unique())
print(f"Classes: {classes}")

In [ ]:
# Armazena os resultados
def storage(classes, score, precision, recall, f1, cm, i):
  results = []

  results.append({
    'Accuracy': score,
    'Precision': precision,
    'Recall': recall,
    'F-measure': f1
  })

  cm_df = pd.DataFrame(cm)

  # Cria um DataFrame com os resultados
  results_df = pd.DataFrame(results)

  classes_df = pd.DataFrame(classes)

  combined_df = pd.concat([results_df, classes_df], axis = 1)
  combined_df = pd.concat([combined_df, cm_df], axis = 1)


  # Salva o DataFrame em um arquivo Excel
  combined_df.to_excel('resultados_experimento_divisao10' + model + str(i) + '.xlsx', index=False)

In [ ]:
for train_index, test_index in rskf.split(X, y):
    # dividi os dados em treino e teste
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # usa o pipe do imblearn para aplicar o smote apenas nos subjconjuntos de treinamento
    pipeline.fit(X_train, y_train)

    # usa os subconjuntos de teste para testar e obter as metricas
    score = pipeline.score(X_test, y_test)                                      # acuracia

    print("Session accuracy:", score)

    # matrizes de confusao
    y_pred = pipeline.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    precision = precision_score(y_test, y_pred, average='weighted')             # precisao
    recall = recall_score(y_test, y_pred, average='weighted')                   # recall
    f1 = f1_score(y_test, y_pred, average='weighted')                           # f-measure

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F-measure: ", f1)

    print(cm, "\n")